# Import Libraries

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import string
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# from models.train_classifier import *

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [2]:
FEATURE_COLUMNS =['message']
TARGET_COLUMNS = [
    'related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 
    'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 
    'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 
    'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 
    'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
    'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 
    'other_weather', 'direct_report']

STOPWORDS_SET = set(stopwords.words('english'))
URL_REGEX = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
URL_PLACE_HOLDER = "urlplaceholder"



def load_data(db_filepath, table_name, feature_columns, target_columns):
        engine = create_engine(db_filepath)
        # Create a dataframe from the engine
        df = pd.read_sql_table(table_name, engine)
        """
        Right now, the way I am importing the data is creating >1D arrays. These will need to be 1D arrays
        so that the CountVectorizer function works. I need to figure out how to do this
        """
        feature_values = df[feature_columns].copy().values
        target_values = df[target_columns].copy().values
        return feature_values, target_values

def tokenize(text):
    # Detect and replace URLs
    text = re.sub(URL_REGEX, URL_PLACE_HOLDER, text)
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [token for token in tokens if token not in STOPWORDS_SET]
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = [lemmatizer.lemmatize(token.lower().strip()) for token in tokens]
    return cleaned_tokens

    
#Load the data
feature_values, target_values = load_data(
                    'sqlite:///data/02_stg//stg_disaster_response.db',
                    'stg_disaster_response',
                    FEATURE_COLUMNS, 
                    TARGET_COLUMNS
                    )

#Print the name, type and shape of the splits from load_data
print("feature_values: ", type(feature_values), feature_values.shape)
print("target_values: ", type(target_values), target_values.shape)

# perform train test #
feature_train, feature_test, target_train, target_test = train_test_split(feature_values, target_values, test_size=0.33, random_state=42)

#Print the name, type and shape of the splits from train test split
print("feature_train: ", type(feature_train), feature_train.shape)
print("feature_test: ", type(feature_test), feature_test.shape)
print("target_train: ", type(target_train), target_train.shape)
print("target_test: ", type(target_test), target_test.shape)

# # Instantiate transformers and classifier
# vect = CountVectorizer(tokenizer=tokenize)
# tfidf = TfidfTransformer(smooth_idf=False)
# clf = RandomForestClassifier()

# # Fit and/or transform each to the training data
# feature_train_counts = vect.fit_transform(feature_train)
# feature_train_tfidf = tfidf.fit_transform(feature_train_counts)

# # Fit or train the classifier
# clf.fit(feature_train_tfidf, target_train)


feature_values:  <class 'numpy.ndarray'> (26216, 1)
target_values:  <class 'numpy.ndarray'> (26216, 36)
feature_train:  <class 'numpy.ndarray'> (17564, 1)
feature_test:  <class 'numpy.ndarray'> (8652, 1)
target_train:  <class 'numpy.ndarray'> (17564, 36)
target_test:  <class 'numpy.ndarray'> (8652, 36)
